In [56]:
import numpy as np
import pandas as pd

In [57]:
winedata = pd.read_csv("RedWineQualityData.csv")

In [58]:
winedata

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [59]:
winedata.info() #So it seems as if the quality of the wine is an integer, which leads me to favor the idea
#of classification over regression as regression may result in a float prediction.
quality = winedata[['quality']]
winefactors = winedata.drop('quality', axis = "columns")
winefactors.info()
#All of this data is filled in and has no missing components so we dont have to worry about Imputing
#however now lets try regression and then try normalization to regression, and then try classification
#to see the best possible results.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 11 columns):
 #   Column        

In [60]:
#Lets try a quick linear regression model and see how it performs lol
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
lrmod = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(winefactors, quality, test_size = 0.2)
lrmod.fit(X_train, y_train)
predictions = lrmod.predict(X_test)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predictions)
#With mean absolute error metric we see we are about .5 off on average which is about 95% accuracy
errorperc = .5/10
print(errorperc)

0.05


In [61]:
#Lets try ridge regression
from sklearn.linear_model import Ridge
rmod = Ridge()
rmod.fit(X_train, y_train)
rmodpredict = rmod.predict(X_test)
mean_absolute_error(y_test, rmodpredict) #We get an even higher error rate of about 

0.49036970037223443

In [62]:
from sklearn.linear_model import Lasso
lmod = Lasso()
lmod.fit(X_train, y_train)
lmodpredict = lmod.predict(X_test)
mean_absolute_error(y_test, lmodpredict) #Lasso performs even more poorly with a 6.8% error
#Seems as if the regular linear regression is most trustworthy linear model, however lets try elasticnet

0.6239008452354926

In [63]:
from sklearn.linear_model import ElasticNet
enmod = ElasticNet(alpha = 1, l1_ratio = .1, normalize = True)
enmod.fit(X_train, y_train)
enmodpredict = enmod.predict(X_test)
mean_absolute_error(y_test, enmodpredict) #This is pretty poor performance relative to the regular lr model
#The regular lr model was able to get an accuracy of 95% while these models are in the realm of 93%

0.6471633111806099

In [64]:
#Lets try linear regression again but with normalization
lrmod2 = LinearRegression()
lrmod2.fit(X_train, y_train)
lrmod2predictions = lrmod.predict(X_test)
mean_absolute_error(y_test, lrmod2predictions) #5% error

0.4882731734240597

In [65]:
#normalize 5.33% error
#Lets turn this into a multiclass classification
quality

,quality
0,5
1,5
2,5
3,6
4,5
...,...
1594,5
1595,6
1596,6
1597,5


In [124]:
#We can say anything in quality that is greater than or equal to 7 is now a 2
#Anything between 4 and 7 is a 1
#before 4 is a 0
poor = [0,1,2,3,4]
average = [5,6,7]
good = [8,9,10]
quality.replace(to_replace = poor, value = 0, inplace= True)
quality.replace(to_replace = average, value = 1, inplace = True)
quality.replace(to_replace = good, value = 2, inplace=True)

C:\Users\tun84049\Anaconda3\lib\site-packages\pandas\core\frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [125]:
#Ok now that we have a classification setup in place, lets see the accuracy we can get with a rfc

In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
xclassiftrain, xclassiftest, yclassiftrain, yclassiftest = train_test_split(winefactors, quality, test_size=0.2)
rfc = RandomForestClassifier()
rfc.fit(xclassiftrain, yclassiftrain)
rfcpredictions = rfc.predict(xclassiftest)
accuracy_score(yclassiftest, rfcpredictions) #We get an accuracy of about 96.5%
# from sklearn.metrics import recall_score, precision_score
# precision_score(yclassiftest, rfcpredictions, average = 'weighted') #We get a precision score of 96.5%
#Seems as if we are obtaining an accuracy score of exactly 1 which is perfect!

C:\Users\tun84049\AppData\Local\Temp/ipykernel_5932/721049196.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(xclassiftrain, yclassiftrain)


1.0

In [113]:
#Although we got a precision score of 96.5%, this heavily changes when we make it weighted due to the fact
#that there is an uneven distribution amongst amount of times we see the different labels. It turns out
#that there are a lot more 1's than 0's and 2's, which means if we mess up more on the 1's it will impact 
#us more, which is likely why our score got lowered a good bit.

In [119]:
# #Lets find best threshhold score for precision vs recall trade off
# from sklearn.metrics import f1_score
# def properThreshold(yclassiftest, rfcpredictions):
#     best_threshhold = 0
#     best_score=0
#     currentscore = 0
#     for i in np.arange(.01, .5, 0.01):
#         currentscore = f1_score(yclassiftest, np.array(rfcpredictions)>i, average = "weighted")
#         if currentscore > best_score:
#             best_threshhold = i
#             best_score = currentscore
        
#     return best_threshhold, best_score
# print(properThreshold(yclassiftest, rfcpredictions))
        

(0.01, 0.006230529595015576)


In [132]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
newrfc = RandomForestClassifier()
params = {
    'n_estimators':[100,200,300],
    'criterion':['gini', 'entropy'],
    'min_samples_split':[2,10,20]
}
X_vals = winefactors
Y_vals = quality
clf = GridSearchCV(newrfc, params, cv = 5)
clf.fit(X_vals, Y_vals)
clfresults = pd.DataFrame(clf.cv_results_)
clfresults

C:\Users\tun84049\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\tun84049\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\tun84049\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\tun84049\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.094191,0.001089,0.008180,0.000398,gini,2,100,"{'criterion': 'gini', 'min_samples_split': 2, ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.190083,0.004211,0.014764,0.000401,gini,2,200,"{'criterion': 'gini', 'min_samples_split': 2, ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.283847,0.007459,0.020748,0.000398,gini,2,300,"{'criterion': 'gini', 'min_samples_split': 2, ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.096348,0.001849,0.008574,0.000790,gini,10,100,"{'criterion': 'gini', 'min_samples_split': 10,...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.187346,0.003542,0.014360,0.000784,gini,10,200,"{'criterion': 'gini', 'min_samples_split': 10,...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
5,0.285440,0.012302,0.021536,0.000796,gini,10,300,"{'criterion': 'gini', 'min_samples_split': 10,...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
6,0.097546,0.003541,0.008180,0.000748,gini,20,100,"{'criterion': 'gini', 'min_samples_split': 20,...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
7,0.189493,0.002595,0.014352,0.000488,gini,20,200,"{'criterion': 'gini', 'min_samples_split': 20,...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
8,0.286239,0.007900,0.021148,0.000408,gini,20,300,"{'criterion': 'gini', 'min_samples_split': 20,...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
9,0.095136,0.002056,0.007790,0.000405,entropy,2,100,"{'criterion': 'entropy', 'min_samples_split': ...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [133]:
clf.best_estimator_

RandomForestClassifier()

In [134]:
clf.best_score_

1.0